#### importy

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.naive_bayes import GaussianNB
from sklearn import decomposition
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

#### załadowanie danych

In [2]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

#### wytrenowanie naiwnym klasyfikatorem

In [3]:
bayes = GaussianNB()
prediction = bayes.fit(X_train, y_train)

#### skuteczności

In [4]:
print("trening :", prediction.score(X_train, y_train))
print("test :", prediction.score(X_test, y_test))

trening : 0.5568775510204081
test : 0.558952380952381


#### PCA

In [5]:
pca = decomposition.PCA(n_components=X_train.shape[1])
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=784, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

#### liczba skłądowych - pierwsza liczba dla której współczynnik wyjaśnianej zmienności przekracza 90%

In [6]:
total_sum = np.cumsum(pca.explained_variance_ratio_)  >= .9
components = np.argmax(total_sum >= 0.90) + 1
# print(total_sum)

#### MINST przekształcony przez PCA

In [7]:
pca_best = decomposition.PCA(n_components=components)
pca_best.fit(X_train)
X_train_pca = pca_best.transform(X_train)
X_test_pca = pca_best.transform(X_test)

#### dokładność zbioru treningowego i testowego

In [8]:
bayes_pca = GaussianNB()
model_prediction = bayes_pca.fit(X_train_pca, y_train)
print("trening: ", model_prediction.score(X_train_pca, y_train))
print("test: ", model_prediction.score(X_test_pca, y_test))

trening:  0.8732040816326531
test:  0.8724285714285714


#### lasy losowe

In [9]:
# przy max_iter=32 dla regresji logistycznej otrzymywałem ostrzeżenie o przekroczeniu liczby iteracji 
# dla takiego zbioru przy tych danych
rfc = RandomForestClassifier(random_state=0, n_estimators=200, max_leaf_nodes=15).fit(X_train, y_train)
rfc_train_acc = rfc.score(X_train, y_train)
rfc_test_acc = rfc.score(X_test, y_test)
print('trening:', rfc_train_acc)
print('test:', rfc_test_acc)

trening: 0.8167142857142857
test: 0.8132380952380952


#### lasy losowe z PCA

In [10]:
rfc = RandomForestClassifier(random_state=0, n_estimators=200, max_leaf_nodes=15).fit(X_train_pca, y_train)
rfc_train_acc = rfc.score(X_train_pca, y_train)
rfc_test_acc = rfc.score(X_test_pca, y_test)
print('trening:', rfc_train_acc)
print('test:', rfc_test_acc)

trening: 0.797265306122449
test: 0.7917619047619048


Po przekształceniu MINST przez PCA skuteczność zarówno dla danych testowych i treningowych wzrasta znacząco w porównaniu do samego wytrenowania przez naiwny klasyfikator: ~56% -> 87%
Zastosowanie PCA dla lasów losowych w powyższym przypadku zmniejsza ich skuteczność, w zamian proces nauczania zajmuje mniej czasu. Spadek z ~81% do ~80%.